In [1]:
## From Deep Learning class implementation

In [2]:
from EHRDataloader import EHRdataFromPickles, EHRdataloader

## 1. Load Dataset

This part of the code loads the dataset, we use the EHRDataLoader.py The initial code could be found: https://github.com/ZhiGroup/pytorch_ehr

In [6]:
print('1 file found. Data will be split into train, validation and test.')
data = EHRdataFromPickles(root_dir = '../data/', 
                      file = 'toy.train', 
                      sort= False,
                      test_ratio = 0.2, 
                      valid_ratio = 0.1,
                      model='RNN') #No sort before splitting

# Dataloader splits
train, test, valid = data.__splitdata__() #this time, sort is true
# can comment out this part if you dont want to know what's going on here
# print(colored("\nSee an example data structure from training data:", 'green'))
# print(data.__getitem__(35, seeDescription = True))

1 file found. Data will be split into train, validation and test.


In [7]:
## Load the labels

In [ ]:
## Get the patients labels, where 1: heart failure and 0: no heart failure
labels=[]
for ii in range(len(train)):
    label=train[ii][1]
    labels.append(label)

In [ ]:
## Distribution of the labels
from collections import Counter
Counter(labels)

In [10]:
## print sizes of train test and valid datasets

In [11]:
len(train),len(test),len(valid)

(7000, 2000, 1000)

## 2. Sample of dataset

In [12]:
## example dataset for patient 0 and visit 0
patient=2
visit=0
print("Patient ID:", train[patient][0])
print("Heart Failure:", train[patient][1])
print("# of visits:", len(train[patient][2]))

print(f' list of visit_time (since last time): {train[patient][2][visit][0]}')
print(f' list of codes corresponding to visit: {train[patient][2][visit][1]}')



Patient ID: 1873
Heart Failure: 1
# of visits: 141
 list of visit_time (since last time): [17739]
 list of codes corresponding to visit: [3886, 6080, 6595, 4148, 373, 7530, 18327, 13971, 16273, 8777, 5682, 342, 4479, 8031, 11130, 19166, 7315, 12275, 12988, 8735, 647, 19515, 11728, 11449, 17860, 15215, 17746, 14643, 5422, 17009, 16438, 16686]


## 3. Preprocess Data for Training

This part of the code transforms the data which has the format described above

In [13]:
import numpy as np

- pids: contains the patient ids
- vids: contains a list of visit ids for each patient
- hfs: contains the heart failure label (0: normal, 1: heart failure) for each patient
- seqs: contains a list of visit (in ICD9 codes) for each patient


In [14]:
## Prepare the data for train set
aux=[len(train[ii][2]) for ii in range(len(train))]
vids=[list(np.arange(ii)) for ii in aux]
hfs=[train[ii][1] for ii in range(len(train))]

seqs=[]
for ii in range(len(train)):
    patient=[]
    for visit in range(len(train[ii][2])):
        patient.append(train[ii][2][visit][1])
    seqs.append(patient)
    
pids=[train[ii][0] for ii in range(len(train))]
assert len(pids) == len(vids) == len(hfs) == len(seqs)


In [15]:
## Prepare the data for val set
aux_val=[len(valid[ii][2]) for ii in range(len(valid))]
vids_val=[list(np.arange(ii)) for ii in aux_val]
hfs_val=[valid[ii][1] for ii in range(len(valid))]

seqs_val=[]
for ii in range(len(valid)):
    patient=[]
    for visit in range(len(valid[ii][2])):
        patient.append(valid[ii][2][visit][1])
    seqs_val.append(patient)
    
pids_val=[valid[ii][0] for ii in range(len(valid))]
assert len(pids_val) == len(vids_val) == len(hfs_val) == len(seqs_val)

In [16]:
## Prepare the data for test set

aux_test=[len(test[ii][2]) for ii in range(len(test))]
vids_test=[list(np.arange(ii)) for ii in aux_test]
hfs_test=[test[ii][1] for ii in range(len(test))]

seqs_test=[]
for ii in range(len(test)):
    patient=[]
    for visit in range(len(test[ii][2])):
        patient.append(test[ii][2][visit][1])
    seqs_test.append(patient)
    
pids_test=[test[ii][0] for ii in range(len(test))]

assert len(pids_test) == len(vids_test) == len(hfs_test) == len(seqs_test)

In [17]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    
    def __init__(self, seqs, hfs):
        self.x = seqs
        self.y = hfs
    
    def __len__(self):
    
        # your code here
        return len(self.x)
    
    def __getitem__(self, index):
        
        # your code here
        return self.x[index],self.y[index]
        
train_dataset = CustomDataset(seqs, hfs)

test_dataset = CustomDataset(seqs_test, hfs_test)

val_dataset= CustomDataset(seqs_val, hfs_val)


In [18]:
def collate_fn(data):
    """
    TODO: Collate the the list of samples into batches. For each patient, you need to pad the diagnosis
        sequences to the sample shape (max # visits, max # diagnosis codes). The padding infomation
        is stored in `mask`.
    
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (# patiens) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            for ii,code in enumerate(visit):
                
                x[i_patient,j_visit,ii]=code
                masks[i_patient,j_visit,ii]=1
             

        for j_visit, visit in enumerate(patient[::-1]):
            for ii,code in enumerate(visit):
                rev_x[i_patient,j_visit,ii]=code
                rev_masks[i_patient,j_visit,ii]=1
    
    return x, masks, rev_x, rev_masks, y

In [19]:
# This is a batch sample from the train set
import torch
from torch.utils.data import DataLoader

loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_fn)
loader_iter = iter(loader)
x, masks, rev_x, rev_masks, y = next(loader_iter)


In [20]:
## 154 max number of visits
## 49 max number of diagnosis codes

In [21]:
x.shape, masks.shape, rev_x.shape, rev_masks.shape, y.shape 

(torch.Size([32, 154, 49]),
 torch.Size([32, 154, 49]),
 torch.Size([32, 154, 49]),
 torch.Size([32, 154, 49]),
 torch.Size([32]))

In [22]:

print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))
print("Length of test dataset:", len(test_dataset))


Length of train dataset: 7000
Length of val dataset: 1000
Length of test dataset: 2000


In [23]:
from torch.utils.data import DataLoader

def load_data(train_dataset, val_dataset,test_dataset, collate_fn):
    
    '''
    TODO: Implement this function to return the data loader for  train and validation dataset. 
    Set batchsize to 32. Set `shuffle=True` only for train dataloader.
    
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, val_loader: train and validation dataloaders
    
    Note that you need to pass the collate function to the data loader `collate_fn()`.
    '''
    
    batch_size = 64
    # your code here
    train_loader=DataLoader(train_dataset,collate_fn=collate_fn,batch_size=batch_size,shuffle=True)
    val_loader=DataLoader(val_dataset,collate_fn=collate_fn,batch_size=batch_size,shuffle=False)
    test_loader=DataLoader(test_dataset,collate_fn=collate_fn,batch_size=batch_size,shuffle=False)
    
    
    
    return train_loader, val_loader,test_loader

## We get the train, val and test Data Loaders
train_loader, val_loader,test_loader = load_data(train_dataset, val_dataset,test_dataset, collate_fn)

## 4. Train RETAIN Model

Training the model, note that we are not using any pre-trained model.

In [24]:
import torch.nn as nn

In [25]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """
        Define the linear layer `self.a_att` for alpha-attention using `nn.Linear()`;
        
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        """
        TODO: Implement the alpha attention.
        
        Arguments:
            g: the output tensor from RNN-alpha of shape (batch_size, seq_length, hidden_dim) 
        
        Outputs:
            alpha: the corresponding attention weights of shape (batch_size, seq_length, 1)
            
        HINT: consider `torch.softmax`
        """
        
        alpha=torch.softmax(self.a_att(g),dim=1)
        return alpha

In [26]:
class BetaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        """
        Define the linear layer `self.b_att` for beta-attention using `nn.Linear()`;
        
        Arguments:
            hidden_dim: the hidden dimension
        """
        
        self.b_att = nn.Linear(hidden_dim, hidden_dim)


    def forward(self, h):
        """
        TODO: Implement the beta attention.
        
        Arguments:
            h: the output tensor from RNN-beta of shape (batch_size, seq_length, hidden_dim) 
        
        Outputs:
            beta: the corresponding attention weights of shape (batch_size, seq_length, hidden_dim)
            
        HINT: consider `torch.tanh`
        """
        
        # your code here
        beta=torch.tanh(self.b_att(h))
        return beta

In [27]:
def attention_sum(alpha, beta, rev_v, rev_masks):
    """
    TODO: mask select the hidden states for true visits (not padding visits) and then
        sum the them up.

    Arguments:
        alpha: the alpha attention weights of shape (batch_size, seq_length, 1)
        beta: the beta attention weights of shape (batch_size, seq_length, hidden_dim)
        rev_v: the visit embeddings in reversed time of shape (batch_size, # visits, embedding_dim)
        rev_masks: the padding masks in reversed time of shape (# visits, batch_size, # diagnosis codes) (bath_size,# visits, #diagnosis codes)

    Outputs:
        c: the context vector of shape (batch_size, hidden_dim)
        
    NOTE: Do NOT use for loop.
    """
    
    out=torch.unsqueeze(torch.max(rev_masks,dim=2).values,-1) * rev_v
    
    out=out*alpha*beta
    
    out=torch.sum(out,dim=1)
    
    return out

In [28]:
def sum_embeddings_with_mask(x, masks):
    """
    Mask select the embeddings for true visits (not padding visits) and then sum the embeddings for each visit up.

    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    
    x = x * masks.unsqueeze(-1)
    x = torch.sum(x, dim = -2)
    return x

In [29]:
## We get the total number of unique codes
all_codes=[]
for patient in seqs:
    for visit in patient:
        all_codes.extend(visit)
        

In [30]:
class RETAIN(nn.Module):
    
    def __init__(self, num_codes, embedding_dim=128):
        super().__init__()
        # Define the embedding layer using `nn.Embedding`. Set `embDimSize` to 128.
        self.embedding = nn.Embedding(num_codes, embedding_dim)
        # Define the RNN-alpha using `nn.GRU()`; Set `hidden_size` to 128. Set `batch_first` to True.
        self.rnn_a = nn.GRU(embedding_dim, embedding_dim, batch_first=True)
        # Define the RNN-beta using `nn.GRU()`; Set `hidden_size` to 128. Set `batch_first` to True.
        self.rnn_b = nn.GRU(embedding_dim, embedding_dim, batch_first=True)
        # Define the alpha-attention using `AlphaAttention()`;
        self.att_a = AlphaAttention(embedding_dim)
        # Define the beta-attention using `BetaAttention()`;
        self.att_b = BetaAttention(embedding_dim)
        # Define the linear layers using `nn.Linear()`;
        self.fc = nn.Linear(embedding_dim, 1)
        # Define the final activation layer using `nn.Sigmoid().
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, masks, rev_x, rev_masks):
        """
        Arguments:
            rev_x: the diagnosis sequence in reversed time of shape (# visits, batch_size, # diagnosis codes)
            rev_masks: the padding masks in reversed time of shape (# visits, batch_size, # diagnosis codes)

        Outputs:
            probs: probabilities of shape (batch_size)
        """
        # 1. Pass the reversed sequence through the embedding layer;
        rev_x = self.embedding(rev_x)
        # 2. Sum the reversed embeddings for each diagnosis code up for a visit of a patient.
        rev_x = sum_embeddings_with_mask(rev_x, rev_masks)
        # 3. Pass the reversed embegginds through the RNN-alpha and RNN-beta layer separately;
        g, _ = self.rnn_a(rev_x)
        h, _ = self.rnn_b(rev_x)
        # 4. Obtain the alpha and beta attentions using `AlphaAttention()` and `BetaAttention()`;
        alpha = self.att_a(g)
        beta = self.att_b(h)
        # 5. Sum the attention up using `attention_sum()`;
        c = attention_sum(alpha, beta, rev_x, rev_masks)
        # 6. Pass the context vector through the linear and activation layers.
        logits = self.fc(c)
        probs = self.sigmoid(logits)
        return probs.squeeze()
    

In [31]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


def eval(model, val_loader):
    
    """
    Evaluate the model.
    
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    REFERENCE: checkout https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    """
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        y_logit = model(x, masks, rev_x, rev_masks)
        """
        TODO: obtain the predicted class (0, 1) by comparing y_logit against 0.5, 
              assign the predicted class to y_hat.
        """
        y_hat = (y_logit>0.5).int()
        # your code here
#         raise NotImplementedError
        y_score = torch.cat((y_score,  y_logit.detach().to('cpu')), dim=0)
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
    return p, r, f, roc_auc

In [32]:
def train(model, train_loader, val_loader,test_loader, n_epochs):
    """
    Train the model.
    
    Arguments:
        model: the RNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
    """
    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x, masks, rev_x, rev_masks)
            """ 
            TODO: calculate the loss using `criterion`, save the output to loss.
            """
            loss = criterion(y_hat,y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'.format(epoch+1, p, r, f, roc_auc))
        p, r, f, roc_auc = eval(model, test_loader)
        print('Epoch: {} \t Test p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'.format(epoch+1, p, r, f, roc_auc))
        print('\n')
    return round(roc_auc, 2)

## 5. Results RETAIN MODEL

Here you will find the table of the results for the RNN GRU

In [33]:
# load the model
retain = RETAIN(num_codes = len(set(all_codes))+1,embedding_dim=128)
# load the loss function
criterion = nn.BCELoss()
# load the optimizer
optimizer = torch.optim.Adam(retain.parameters(), lr=0.001)

# optimizer = torch.optim.RMSprop(retain.parameters(), lr=0.01)
# optimizer = torch.optim.SGD(retain.parameters(), 
#                       lr=0.001,
#                       weight_decay=0.9)

n_epochs = 10
train(retain, train_loader, val_loader,test_loader, n_epochs)

Epoch: 1 	 Training Loss: 0.693934
Epoch: 1 	 Validation p: 0.53, r:0.41, f: 0.46, roc_auc: 0.52
Epoch: 1 	 Test p: 0.52, r:0.38, f: 0.44, roc_auc: 0.51


Epoch: 2 	 Training Loss: 0.546473
Epoch: 2 	 Validation p: 0.54, r:0.53, f: 0.54, roc_auc: 0.53
Epoch: 2 	 Test p: 0.51, r:0.52, f: 0.52, roc_auc: 0.50


Epoch: 3 	 Training Loss: 0.155120
Epoch: 3 	 Validation p: 0.53, r:0.53, f: 0.53, roc_auc: 0.53
Epoch: 3 	 Test p: 0.52, r:0.52, f: 0.52, roc_auc: 0.51


Epoch: 4 	 Training Loss: 0.034107
Epoch: 4 	 Validation p: 0.54, r:0.53, f: 0.54, roc_auc: 0.53
Epoch: 4 	 Test p: 0.52, r:0.51, f: 0.51, roc_auc: 0.50


Epoch: 5 	 Training Loss: 0.014557
Epoch: 5 	 Validation p: 0.54, r:0.54, f: 0.54, roc_auc: 0.53
Epoch: 5 	 Test p: 0.51, r:0.51, f: 0.51, roc_auc: 0.50


Epoch: 6 	 Training Loss: 0.009038
Epoch: 6 	 Validation p: 0.54, r:0.54, f: 0.54, roc_auc: 0.53
Epoch: 6 	 Test p: 0.51, r:0.51, f: 0.51, roc_auc: 0.50


Epoch: 7 	 Training Loss: 0.006316
Epoch: 7 	 Validation p: 0.54, r:0.

0.5

In [34]:
# after model training
p, r, f, roc_auc = eval(retain, train_loader)
print(p, r, f, roc_auc)


p, r, f, roc_auc = eval(retain, test_loader)
print(p, r, f, roc_auc)

1.0 1.0 1.0 1.0
0.5126953125 0.5097087378640777 0.5111976630963972 0.5048233410069062


In [35]:
# without model training
retain = RETAIN(num_codes = len(set(all_codes))+1)

p, r, f, roc_auc = eval(retain, train_loader)
print(p, r, f, roc_auc)


p, r, f, roc_auc = eval(retain, test_loader)
print(p, r, f, roc_auc)

0.4987075928917609 0.8792366847052122 0.6364292340995773 0.48944193652851586
0.5 0.525242718446602 0.5123106060606061 0.4896987288559704
